## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


### BUSINESS UNDERSTANDING

##### KEY QUESTIONS FOR MOVIE STUDIO STRATEGY
1. Which movie genres generate the highest revenue? By analyzing the total earnings from the different genres, we can determine which types of films are more profitable and likely to attract large audiences.
2. Does movie rating or critic score correlate with higher box office success? using ratings from Rotten Tomatoes and movie database, we can measure whether higher-rated movies earn more revenue 
3. Does a longer runtime affect rating or revenue? By analyzing the runtimes in the dataset, we can identify which runtimes generate higher earnings for different types of movies.
4. Does the release timing affect the earnings? By examining which months and their associated revenue, we can identify the optimal times to launch films for maximum success.

In [1]:
import pandas as pd
import sqlite3


In [2]:
import sqlite3

# Connect to your IMDB database
conn = sqlite3.connect('im.db')
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)


Tables in the database: []


In [3]:
import os
print(os.path.abspath('im.db'))
print(os.path.getsize('im.db'))


c:\Users\Dell\Documents\flatiron\DataScience\Phase2\project2\Movie-Studio-EDA-Project\im.db
0


In [6]:
import os
print(os.listdir("zippedData"))


['bom.movie_gross.csv', 'bom.movie_gross.csv.gz', 'im.db', 'im.db.zip', 'rt.movie_info.tsv.gz', 'rt.reviews.tsv.gz', 'tmdb.movies.csv.gz', 'tn.movie_budgets.csv.gz']


In [ ]:
import pandas as pd

bom_df = pd.read_csv("zippedData/bom.movie_gross.csv.gz", compression='gzip')
bom_df.head()


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [18]:
import zipfile

zip_path = "zippedData/im.db.zip"
extract_to = "data/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Files after extraction:", os.listdir(extract_to))



Files after extraction: ['im.db']


In [ ]:
conn = sqlite3.connect("data/im.db")
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:", tables)
conn.close()


Tables in the database:             name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [21]:
conn = sqlite3.connect("rt.movie_gross.csv.gz")
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:", tables)
conn.close()

Tables in the database: Empty DataFrame
Columns: [name]
Index: []


In [24]:
import pandas as pd

# Load Rotten Tomatoes movie info
rt_info = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep="\t", compression="gzip")

# Peek at the data
print(rt_info.head())
print(rt_info.columns)


   id                                           synopsis rating  \
0   1  This gritty, fast-paced, and innovative police...      R   
1   3  New York City, not-too-distant-future: Eric Pa...      R   
2   5  Illeana Douglas delivers a superb performance ...      R   
3   6  Michael Douglas runs afoul of a treacherous su...      R   
4   7                                                NaN     NR   

                                 genre          director  \
0  Action and Adventure|Classics|Drama  William Friedkin   
1    Drama|Science Fiction and Fantasy  David Cronenberg   
2    Drama|Musical and Performing Arts    Allison Anders   
3           Drama|Mystery and Suspense    Barry Levinson   
4                        Drama|Romance    Rodney Bennett   

                            writer  theater_date      dvd_date currency  \
0                   Ernest Tidyman   Oct 9, 1971  Sep 25, 2001      NaN   
1     David Cronenberg|Don DeLillo  Aug 17, 2012   Jan 1, 2013        $   
2          

In [27]:
import pandas as pd

# Use encoding='latin-1' to avoid UnicodeDecodeError
rt_reviews = pd.read_csv(
    "zippedData/rt.reviews.tsv.gz",
    sep="\t",
    compression="gzip",
    encoding="latin-1"
)

# Peek at the data
rt_reviews.head()



,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


# Loading Box Office Mojo data

In [28]:
import pandas as pd

# Load the gzipped CSV
bom_df = pd.read_csv("data/bom.movie_gross.csv.gz", compression='gzip')

# Peek at the data
bom_df.head()


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


# Connect to IMDB database and load tables

In [29]:
import sqlite3

conn = sqlite3.connect("data/im.db")

# List tables to see what’s available
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)


            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


# Merging IMDB basics and ratings


# Checking the IMBD database

In [37]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data/im.db")
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:", tables)
conn.close()


Tables in the database:             name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [41]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("data/im.db")

# Read the tables into Python
movie_basics = pd.read_sql("SELECT * FROM movie_basics;", conn)
movie_ratings = pd.read_sql("SELECT * FROM movie_ratings;", conn)

# Close the connection
conn.close()

# Check that they loaded correctly
print(movie_basics.head())
print(movie_ratings.head())


    movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3  tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4  tt0100275         The Wandering Soap Opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  
0        2013            175.0    Action,Crime,Drama  
1        2019            114.0       Biography,Drama  
2        2018            122.0                 Drama  
3        2018              NaN          Comedy,Drama  
4        2017             80.0  Comedy,Drama,Fantasy  
     movie_id  averagerating  numvotes
0  tt10356526            8.3        31
1  tt10384606            8.9       559
2   tt1042974            6.4        20
3   tt1043726            4.2     50352
4   tt106024

In [45]:
print(movie_basics.columns)
print(movie_ratings.columns)


Index(['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres'],
      dtype='object')
Index(['movie_id', 'averagerating', 'numvotes'], dtype='object')


## Merged IMDB basics and ratings

In [47]:
imdb_df = pd.merge(movie_basics, movie_ratings, on='movie_id', how='left')
imdb_df.head()


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


## Merge box office mojo data

#### Matching by movie title

In [48]:
# Lowercase for merging
bom_df['title_lower'] = bom_df['title'].str.lower()
imdb_df['primary_title_lower'] = imdb_df['primary_title'].str.lower()

# Merge
full_df = pd.merge(bom_df, imdb_df, left_on='title_lower', right_on='primary_title_lower', how='left')


## Merge rotten tomatoes

In [52]:
rt_info.columns['id_lower'] = rt_info['id_title'].str.lower()
full_df = pd.merge(full_df, rt_info, left_on='title_lower', right_on='movie_title_lower', how='left')
print(rt_info.columns)

KeyError: 'id_title'

In [54]:
# Ensure the ID columns are the same type
im.db_df['id'] = im.db_df['id'].astype(str)
rt_info['id'] = rt_info['id'].astype(str)

# Merge Rotten Tomatoes info into your IMDb data
full_df = pd.merge(imdb_df, rt_info, on='id', how='left')

# Check result
full_df.head()


NameError: name 'im' is not defined